# 🏦 JP Morgan Quantitative Research
### Task 3 – Credit Risk Analysis & Expected Loss Modeling

### Business Problem

Retail banking loans generate revenue through interest payments,
but also expose the bank to **credit risk** — the risk that a borrower
will default on their loan.

The risk team wants to:
- Predict the **Probability of Default (PD)** for each borrower
- Estimate the **Expected Loss (EL)** on loans
- Ensure sufficient capital is held to absorb potential losses

We are provided with historical loan data and borrower characteristics
to build a **prototype predictive model**.


### Key Credit Risk Concepts

- **Probability of Default (PD)**  
  The likelihood that a borrower will fail to repay the loan.

- **Loss Given Default (LGD)**  
  Percentage of exposure lost if a default occurs.  
  (Given: Recovery Rate = 10% → LGD = 90%)

- **Exposure at Default (EAD)**  
  Outstanding loan amount.

- **Expected Loss (EL)**  
  EL = PD × LGD × EAD

This framework is standard under Basel risk regulations.


In [38]:
import pandas as pd
import numpy as np

In [40]:
df = pd.read_csv("Customer Loan Data.csv")
df.head()


,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               10000 non-null  int64  
 1   credit_lines_outstanding  10000 non-null  int64  
 2   loan_amt_outstanding      10000 non-null  float64
 3   total_debt_outstanding    10000 non-null  float64
 4   income                    10000 non-null  float64
 5   years_employed            10000 non-null  int64  
 6   fico_score                10000 non-null  int64  
 7   default                   10000 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 625.1 KB


### Dataset Overview

We have loan-level customer data with the following fields:

- customer_id: Unique customer identifier
- credit_lines_outstanding: Number of active credit lines
- loan_amt_outstanding: Current loan exposure (EAD)
- total_debt_outstanding: Total debt across all products
- income: Annual income of borrower
- years_employed: Employment stability indicator
- fico_score: Credit score (key risk driver)
- default: Target variable (1 = default, 0 = non-default)

This dataset is well-structured with:
- No missing values
- Clear numerical predictors
- Binary default label


### 📥 Step 2: Prepare Features & Target

In [46]:
X = df.drop(columns=['customer_id', 'default'])
y = df['default']


In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)


In [50]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [52]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced'
)

log_model.fit(X_train_scaled, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000)

In [54]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = log_model.predict(X_test_scaled)
y_prob = log_model.predict_proba(X_test_scaled)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))


              precision    recall  f1-score   support

           0       1.00      0.99      1.00      2037
           1       0.97      1.00      0.99       463

    accuracy                           1.00      2500
   macro avg       0.99      1.00      0.99      2500
weighted avg       1.00      1.00      1.00      2500

ROC-AUC: 0.9999862161248013


In [57]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['lbfgs']
}

grid = GridSearchCV(
    LogisticRegression(max_iter=1000, class_weight='balanced'),
    param_grid,
    scoring='roc_auc',
    cv=5
)

grid.fit(X_train_scaled, y_train)

best_model = grid.best_estimator_


In [59]:
df['PD'] = best_model.predict_proba(
    scaler.transform(X)
)[:, 1]


In [61]:
df['EAD'] = df['loan_amt_outstanding']


In [63]:
df['LGD'] = 0.45   # Industry average assumption


In [65]:
df['Expected_Loss'] = df['PD'] * df['LGD'] * df['EAD']


In [67]:
import numpy as np

n_simulations = 10000
simulated_losses = []

for _ in range(n_simulations):
    simulated_pd = np.random.beta(
        a=2, b=5, size=len(df)
    )
    loss = np.sum(simulated_pd * df['LGD'] * df['EAD'])
    simulated_losses.append(loss)

simulated_losses = np.array(simulated_losses)


In [68]:
VaR_95 = np.percentile(simulated_losses, 95)
CVaR_95 = simulated_losses[simulated_losses >= VaR_95].mean()

print("Value at Risk (95%):", VaR_95)
print("Conditional VaR (95%):", CVaR_95)


Value at Risk (95%): 5399621.939288806
Conditional VaR (95%): 5412879.575622154


In [69]:
def pricing_engine(pd):
    if pd < 0.05:
        return 8.5
    elif pd < 0.10:
        return 11.0
    elif pd < 0.20:
        return 15.0
    else:
        return 22.0

df['Quoted_Interest_Rate'] = df['PD'].apply(pricing_engine)


In [70]:
df['Loan_Decision'] = np.where(
    (df['PD'] < 0.25) & (df['fico_score'] > 550),
    'Approved',
    'Rejected'
)


In [75]:
final_output = df[[
    'customer_id',
    'fico_score',
    'income',
    'loan_amt_outstanding',
    'PD',
    'Expected_Loss',
    'Quoted_Interest_Rate',
    'Loan_Decision'
]]

final_output.head()


,customer_id,fico_score,income,loan_amt_outstanding,PD,Expected_Loss,Quoted_Interest_Rate,Loan_Decision
0,8153374,605,78039.38546,5221.545193,2.664253e-24,6.260184e-21,8.5,Approved
1,7442532,572,26648.43525,1958.928726,1.000000e+00,8.815179e+02,22.0,Rejected
2,2256073,602,65866.71246,3363.009259,3.006954e-22,4.550586e-19,8.5,Approved
3,4885975,612,74356.88347,4766.648001,7.266033e-25,1.558558e-21,8.5,Approved
4,4700614,631,23448.32631,1345.827718,1.206934e-16,7.309465e-14,8.5,Approved
